In [2]:
import os

In [3]:
%pwd

'c:\\Users\\HP\\Desktop\\chicken-disease-classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\HP\\Desktop\\chicken-disease-classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzipped_data_dir: Path


In [7]:
from chicken_disease.constants import *
from chicken_disease.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzipped_data_dir= config.unzipped_data_dir
        )

        return data_ingestion_config


In [9]:
import os
import urllib.request as request
import zipfile 
from chicken_disease import logger
from chicken_disease.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extracts the zip file to the unzipped_data_dir
        Function returns None
        """
        unzip_path = self.config.unzipped_data_dir
        os.makedirs(unzip_path, exist_ok= True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e

[2025-11-19 18:08:38,098: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-19 18:08:38,134: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-19 18:08:38,148: INFO: common: created directory at : artifacts]
[2025-11-19 18:08:38,150: INFO: common: created directory at : artifacts/data_ingestion]
[2025-11-19 18:08:43,578: INFO: 2811270602: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6600:0A0A:1D8541:433061:691DBA4F
Accept-Ranges: bytes
Date: Wed, 19 Nov 2025 12:38:41 GMT
Via: 1.1 varnish
X-Served-By: cache-sin-wsss1830030-SIN
X

In [13]:
pass